In [1]:
!pip install audformat
!pip install opensmile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.4/167.4 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iso-639: filename=iso_639-0.4.5-py3-none-any.whl size=168841 sha256=8eab40f0de072c99999aa0dc813023a74d361223047974724793012aa76b6d7a
  Stored in directory: /root/.cache/pip/wheels/d8/78/cc/5478ca3b1c3f602eae6f8cdbd78f909c0a0bfa0bbcb5c7771f
Successfully built iso-639
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.2/996.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.6/134.6 kB 9.3 MB/s eta 0:00:00


In [2]:
!wget https://zenodo.org/record/7447302/files/emodb.zip

!unzip emodb.zip

--2023-11-11 21:51:06--  https://zenodo.org/record/7447302/files/emodb.zip
Resolving zenodo.org (zenodo.org)... 188.185.22.33, 188.185.33.206, 188.185.10.78, ...
Connecting to zenodo.org (zenodo.org)|188.185.22.33|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/7447302/files/emodb.zip [following]
--2023-11-11 21:51:07--  https://zenodo.org/records/7447302/files/emodb.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 39981818 (38M) [application/octet-stream]
Saving to: ‘emodb.zip’

emodb.zip           100%[===================>]  38.13M  12.1MB/s    in 3.1s    

2023-11-11 21:51:10 (12.1 MB/s) - ‘emodb.zip’ saved [39981818/39981818]

Archive:  emodb.zip
   creating: emodb/
  inflating: emodb/db.emotion.categories.train.gold_standard.pkl  
   creating: emodb/wav/
  inflating: emodb/wav/14b01Fc.wav   
  inflating: emodb/wav/14a02Ea.wav   
  inflating: emodb/wav/15a02Ta.wav   
  infla

**Loading libraries**

In [3]:
import audformat

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE

import opensmile



**Loading dataset**

using the gold standard train and test data

In [4]:

db = audformat.Database.load('emodb')

df_test = db['emotion.categories.test.gold_standard'].get()

df_train = db['emotion.categories.train.gold_standard'].get()

print(f'samples in train: {df_train.shape[0]}, and test: {df_test.shape[0]}')



samples in train: 304, and test: 231


Using opensmile to extract raw features

In [5]:


smile = opensmile.Smile(
    opensmile.FeatureSet.ComParE_2016,
    opensmile.FeatureLevel.Functionals,
    sampling_rate=16000,
    resample=True,
    num_workers=5,
    verbose=True,
)

X_train = smile.process_index(
    df_train.index,
    root=db.root,

)
X_test = smile.process_index(
    df_test.index,
    root=db.root,

)


**Balancing classes**

In [6]:
# # balance train classes
sm = SMOTE()
X_train, y_train = sm.fit_resample(X_train, df_train.emotion)

Using SVM classifier model

In [7]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))

In [8]:
clf.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [9]:
y_predict= clf.predict(X_test)

In [10]:
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score

In [11]:
accuracy_score(df_test.emotion,y_predict)

0.7402597402597403

In [12]:
confusion_matrix(df_test.emotion,y_predict)

array([[55,  0,  0,  0,  0,  0,  0],
       [ 1, 31,  0,  0,  0,  4,  0],
       [ 3,  3, 14,  1,  4,  1,  0],
       [14,  0,  0, 18,  1,  0,  0],
       [24,  0,  0,  0,  3,  0,  0],
       [ 1,  1,  0,  1,  0, 24,  0],
       [ 0,  1,  0,  0,  0,  0, 26]])